In [1]:
%cd ../

/scratch/project_2010450/thesis


In [2]:
import json
import re
from pathlib import Path

import pandas as pd
import numpy as np

In [3]:
path_dir = Path("/scratch/project_2010450/thesis/logs")

In [4]:
DATASETS = [
    'ml-1m',
    'amazon-digital-music'
]

METRICS = [
    'ndcg@10',
    'precision@10',
    'recall@10',
    'mrr@10',
    'hit@10',
    'map@10',
]

In [5]:
results = []

for path in path_dir.glob("Aug*/tune_result.json"):
    path = str(path)

    # Read file
    with open(path) as f:
        raw = json.load(f)['best_result']

    result = {}
    # Determine model, dataset, evaluation scheme
    model = raw['model']

    dataset = ""
    for name in DATASETS:
        if name in path:
            dataset = name
            break
    assert dataset != ""

    pat = r"usecutoff\_(True|False)"

    match (re.findall(pat, path)[0]):
        case 'True':
            scheme = "Simulated Online"
        case 'False':
            scheme = "Leave-one-out"
        case _:
            raise NotImplementedError()

    print(f"{model}-{dataset}-{scheme}")
    result = {
        **result,
        'model': model,
        'dataset': dataset,
        'evaluation_scheme': scheme
    }

    # Extract metrics
    records = [
        ("val_overall", raw['best_valid_result']),
        ("val_active", raw.get('valid_result_active', None)),
        ("val_inactive", raw.get('valid_result_inactive', None)),
        ("test_overall", raw['test_result']),
        ("test_active", raw.get('test_result_active', None)),
        ("test_inactive", raw.get('test_result_inactive', None)),
    ]

    for tag, record in records:
        if record is not None:
            for metric, val in record.items():
                result[f"{tag}_{metric}"] = val
        else:
            for metric in METRICS:
                result[f"{tag}_{metric}"] = np.nan
    
    results.append(result)


pd.DataFrame.from_records(results).to_csv("metrics.csv", index=None)

HGN-amazon-digital-music-Leave-one-out
HGN-amazon-digital-music-Simulated Online
GRU4Rec-ml-1m-Simulated Online
ItemKNN-amazon-digital-music-Leave-one-out
ENMF-ml-1m-Simulated Online
ENMF-ml-1m-Leave-one-out
HGN-ml-1m-Simulated Online
BPR-ml-1m-Leave-one-out
BPR-ml-1m-Simulated Online
ItemKNN-ml-1m-Leave-one-out
BPR-amazon-digital-music-Simulated Online
GRU4Rec-ml-1m-Leave-one-out
ItemKNN-amazon-digital-music-Simulated Online
GRU4Rec-amazon-digital-music-Leave-one-out
BERT4Rec-amazon-digital-music-Leave-one-out
BERT4Rec-ml-1m-Simulated Online
NPE-ml-1m-Simulated Online
BERT4Rec-amazon-digital-music-Simulated Online
NPE-amazon-digital-music-Simulated Online
GRU4Rec-amazon-digital-music-Simulated Online
NPE-amazon-digital-music-Leave-one-out
NPE-ml-1m-Leave-one-out
BPR-amazon-digital-music-Leave-one-out
BERT4Rec-ml-1m-Leave-one-out
ENMF-amazon-digital-music-Simulated Online
ItemKNN-ml-1m-Simulated Online
ENMF-amazon-digital-music-Leave-one-out
HGN-ml-1m-Leave-one-out
